# Open LLM Models


In [3]:
from dotenv import load_dotenv
import os
if load_dotenv("../.env"):
    GROQ_API_KEY = os.getenv('GROQ_API_KEY')
    HF_API_TOKEN=os.getenv('HF_API_TOKEN')

# What is [OLlama](https://ollama.com/) ?
Ollama allows you run LLMs locally!

Github: https://github.com/ollama/ollama and https://github.com/ollama/ollama-python

## Instructions to Run with Docker

1. Install docker if you don´t have it installed yet: `bash docker.sh`
2. Start docker service: `sudo service docker start`
3. Pull Ollama: `sudo docker run -d -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama`
4. Optionally you can talk directly with the model:  
   a) How to pull a model: `sudo docker exec -it ollama ollama pull llama3`  
   b) How to run a Model: `sudo docker exec -it ollama ollama run llama3`
   

## Instructions to Run in Linux

1. `curl -fsSL https://ollama.com/install.sh | sh`
   - Ollama will be installed automatically as a service
   - `sudo service ollama status`
   - `ollama pull llama3`

2. If you want to see the messages sent to Ollama, you have to run it as a server:
   - Stop ollama service: `sudo service ollama stop`
   - Start Ollama as a service: `ollama server`
   - Open another Linux terminal and type: `ollama pull llama3`

Other popular free model with high performance from Microsoft is `phi3`.
It´s half the size of `Llama3`.

After everything is properly downloaded and running, run the Jupyter cell below

In [4]:
import ollama

#ollama.list()
#ollama.delete(model="name of the model here")
#ollama.pull(model="name of the model here", stream=True)
ollama.show(model="llama3")

{'license': 'META LLAMA 3 COMMUNITY LICENSE AGREEMENT\n\nMeta Llama 3 Version Release Date: April 18, 2024\n“Agreement” means the terms and conditions for use, reproduction, distribution and modification of the Llama Materials set forth herein.\n\n“Documentation” means the specifications, manuals and documentation accompanying Meta Llama 3 distributed by Meta at https://llama.meta.com/get-started/.\n\n“Licensee” or “you” means you, or your employer or any other person or entity (if you are entering into this Agreement on such person or entity’s behalf), of the age required under applicable laws, rules or regulations to provide legal consent and that has legal authority to bind your employer or such other person or entity if you are entering in this Agreement on their behalf.\n\n“Meta Llama 3” means the foundational large language models and software and algorithms, including machine-learning model code, trained model weights, inference-enabling code, training-enabling code, fine-tuning

In [5]:
response = ollama.chat(model='llama3', messages=[
  {
    'role': 'user',
    'content': 'Who do you think is most famous physisist in the world from all times ? Give me a simple and direct answer',
  },
])

response
#Expected execution time: 34s

{'model': 'llama3',
 'created_at': '2024-05-13T21:14:59.122519404Z',
 'message': {'role': 'assistant', 'content': 'Albert Einstein.'},
 'done': True,
 'total_duration': 34132853744,
 'load_duration': 15732009589,
 'prompt_eval_count': 34,
 'prompt_eval_duration': 6602754000,
 'eval_count': 4,
 'eval_duration': 11510559000}

In [6]:

#Streaming responses
stream = ollama.chat(
    model='llama3',
    messages=[{'role': 'user', 'content': 'Do you know C# ? Be short on your answer. No need to provide code example'}],
    stream=True
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)
  
#Expected execution time:  17s

Yes, I'm familiar with C#. I can understand and work with C# code, including its syntax,

KeyboardInterrupt: 

In [ ]:
def chatWithLocalOllama(text:str, model="llama3"):
    return ollama.generate(model=model, prompt=text, stream=True)

In [ ]:
question = "What planet comes after Earth ? Direct answer please"
for chunk in chatWithLocalOllama(question):
    print(chunk['response'], end='', flush=True)
    
# Expected execution time: 5.8s

The next planet after Earth in our solar system is Mars.

In [ ]:
#What if Ollama is hosted somewhere else ? You have to create a custom client pointing to the IP address of the server
from ollama import Client

def chatWithRemoteOllama(text:str, model="llama3"):
    client = Client(host='http://localhost:11434')
    return client.generate(model=model, prompt=text, stream=True)

In [ ]:
question = "What is the Mona Lisa? Answer in the shortest way possible"
for chunk in chatWithRemoteOllama(question):
    print(chunk['response'], end='', flush=True)

#Expected execution time: 10.8s

The Mona Lisa is a famous painting by Leonardo da Vinci, depicting a woman with an enigmatic smile.

## How to remember conversations ?

There is no magic! You have to keep track of the context

In [ ]:
from typing import List

#Yes! It´s possible to have async functions and typed variables in Python too... 

async def chatWithRemoteOllamaAsync(text:List[dict], model="llama3"):
    client = Client(host='http://127.0.0.1:11434')
    return client.chat(model=model, messages=text, stream=True)


#Possible roles: system, user and assistant
#system: this is the persona you want the AI to impersonate. 
#user: this is your question
#assistant: this is the AI reply
messages = [
    {'role':'system','content':'You are Super Mario'},
    {'role':'user', 'content':'Luigi has gonne missing. What is your first thought ? be direct and simple '}
    ]

response = ''
for chunk in await chatWithRemoteOllamaAsync(messages):
    response += chunk['message']['content']
    print(chunk['message']['content'], end='', flush=True)
    
messages = [*messages, {'role':'assistant','content':response}]

messages

"Bowser's behind this, I just know it!"

[{'role': 'system', 'content': 'You are Super Mario'},
 {'role': 'user',
  'content': 'Luigi has gonne missing. What is your first thought ? be direct and simple '},
 {'role': 'assistant', 'content': '"Bowser\'s behind this, I just know it!"'}]

In [ ]:
print('\n')
print('-'*120)
print('2nd iteration\n')

messages = [
    *messages,  # Include the history from the first call
    {'role': 'user', 'content': 'Who Kidnapped him?'}
]

response = ''
for chunk in await chatWithRemoteOllamaAsync(messages):
    response += chunk['message']['content']
    print(chunk['message']['content'], end='', flush=True)

messages = [*messages, {'role':'assistant','content':response}]

#Expected execution time: 18.6s
print("\n")
print("-"*120)
print("Complete history:")
print(messages)



------------------------------------------------------------------------------------------------------------------------
2nd iteration

"Kamek and Bowser! Those no-good Koopa Kingpins are at it again! I've got to rescue my bro and save the Mushroom Kingdom from their evil clutches!"

------------------------------------------------------------------------------------------------------------------------
Complete history:
[{'role': 'system', 'content': 'You are Super Mario'}, {'role': 'user', 'content': 'Luigi has gonne missing. What is your first thought ? be direct and simple '}, {'role': 'assistant', 'content': '"Bowser\'s behind this, I just know it!"'}, {'role': 'user', 'content': 'Who Kidnapped him?'}, {'role': 'assistant', 'content': '"Kamek and Bowser! Those no-good Koopa Kingpins are at it again! I\'ve got to rescue my bro and save the Mushroom Kingdom from their evil clutches!"'}]


# [HuggingFace Transformers](https://github.com/huggingface/transformers)

Hugging Face Transformers is a state-of-the-art machine learning library that provides easy access to pre-trained models for various tasks across different modalities. Here are some key features:

## Modalities Supported

- **Natural Language Processing (NLP)**: Tasks include text classification, named entity recognition, question answering, language modeling, summarization, translation, multiple choice, and text generation.
- **Computer Vision**: Tasks include image classification, object detection, and segmentation.
- **Audio**: Tasks include automatic speech recognition and audio classification.
- **Multimodal**: Tasks include table question answering, optical character recognition, information extraction from scanned documents, video classification, and visual question answering.

## Pretrained Models

Transformers provides APIs and tools to easily download and train state-of-the-art pretrained models. Using pretrained models can reduce your compute costs, carbon footprint, and save you the time and resources required to train a model from scratch.

[100 projects using Transformers](https://github.com/huggingface/transformers/blob/main/awesome-transformers.md)

### Possible Tasks using Pipelines

- "text-generation"
- "conversational"
- "document-question-answering"
- "translation"
- "image-to-text"
- "text-to-audio" (alias "text-to-speech" available)
- "audio-classification"`
- "automatic-speech-recognition"
- "depth-estimation"
- "feature-extraction"
- "fill-mask"
- "image-classification"
- "image-feature-extraction"
- "image-segmentation"
- "image-to-image"
- "token-classification" (alias "ner" available)
- "translation_xx_to_yy"
- "video-classification"
- "visual-question-answering"
- "zero-shot-classification"
- "zero-shot-image-classification"
-"zero-shot-audio-classification"
- "zero-shot-object-detection"

Hugging-Face main homepage: https://huggingface.co/

In [ ]:
from transformers import pipeline

sentiment = pipeline(task="sentiment-analysis", token=HF_API_TOKEN, device="cpu")
results = sentiment("What a lovely day")
results

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998741149902344}]

In [ ]:
from transformers import pipeline
transcriber = pipeline(task="automatic-speech-recognition", token=HF_API_TOKEN, device="cpu")
transcriber(["./Resources/audio1.ogg", "./Resources/audio2_en.ogg"])

No model was supplied, defaulted to facebook/wav2vec2-base-960h and revision 55bb623 (https://huggingface.co/facebook/wav2vec2-base-960h).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebo

[{'text': 'IS TIONTESCIGI HECUIESIMIANTE GUVOS BY THE SABIRSIO PIPLYIN AD GET UP TO I TO THE HAYUN FACE A MUTOBO BY THE TRUSCREVIRTESTUS'},
 {'text': 'THIS MODO CAN ONLY UNDERSTAND ENGLISH'}]

In [ ]:
from transformers import pipeline
transcriber = pipeline(task="automatic-speech-recognition", model="openai/whisper-tiny",token=HF_API_TOKEN, device="cpu")
transcriber(["./Resources/audio1.ogg", "./Resources/audio2_en.ogg"])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


[{'text': ' Este é um teste de reconhecimento de voz para saber se o pipeline gratuito da Rallyon Face é muito bom para transcrever textos.'},
 {'text': ' This model can only understand English.'}]

In [ ]:
from transformers import pipeline
# yes the task is automatically inffered from the model also
vision_classifier = pipeline(
    model="google/vit-base-patch16-224", token=HF_API_TOKEN, device="cpu")
preds = vision_classifier(
    images="https://th.bing.com/th/id/OIP.YjlrCGml5fb7B2pBqtdivQHaE7?rs=1&pid=ImgDetMain"
)
preds

[{'label': 'leopard, Panthera pardus', 'score': 0.9643744230270386},
 {'label': 'jaguar, panther, Panthera onca, Felis onca',
  'score': 0.03195194527506828},
 {'label': 'cheetah, chetah, Acinonyx jubatus',
  'score': 0.0015117806615307927},
 {'label': 'snow leopard, ounce, Panthera uncia',
  'score': 0.0007983926334418356},
 {'label': 'lion, king of beasts, Panthera leo',
  'score': 0.00022117019398137927}]

In [ ]:
img="https://huggingface.co/spaces/impira/docquery/resolve/2359223c1837a7587402bda0f2643382a6eefeab/invoice.png"
initial_questions = [
    "What is the total price of the invoice ?",
    "What is the 2nd item description ?",
    "What is the invoice number ?",
]

more_questions = [
    "What is the Due date?",
    "What is the name of the company who created this invoice?",
    "What is the name of the customer who paid this ?",
    "What is the address of the recipient ?",
    "What is the complete address under SHIP TO ?"
]

image_feature_extraction = pipeline(model="impira/layoutlm-document-qa")
data = lambda i: {"image": img, "question": initial_questions[i]}
print('-'*120)
for i in range(3):
    print(image_feature_extraction(data(i)))
    
# Imagine this is a very large dataset . With yield you defer the execution of the function until it is needed
def large_data():
    for i in range(5):
        yield {"image": img, "question": more_questions[i]}

print('-'*120)
for out in image_feature_extraction(large_data()):
    print(out)
        

------------------------------------------------------------------------------------------------------------------------
[{'score': 0.6036366820335388, 'answer': '$154.06', 'start': 75, 'end': 75}]
[{'score': 0.9854755401611328, 'answer': 'Newset of pedal arms', 'start': 57, 'end': 60}]
[{'score': 0.672887921333313, 'answer': 'us-001', 'start': 16, 'end': 16}]
------------------------------------------------------------------------------------------------------------------------
[{'score': 0.9999239444732666, 'answer': '26/02/2019', 'start': 42, 'end': 42}]
[{'score': 0.9997648000717163, 'answer': 'East Repair Inc.', 'start': 1, 'end': 3}]
[{'score': 0.9997523427009583, 'answer': 'John Smith', 'start': 17, 'end': 18}]
[{'score': 0.2696015536785126, 'answer': 'John Smith', 'start': 17, 'end': 18}]
[{'score': 0.751903772354126, 'answer': 'John Smith', 'start': 17, 'end': 18}]


In [ ]:
from transformers import pipeline
text_generation = pipeline("text-generation", model="openai-community/gpt2", token=HF_API_TOKEN, device="cpu")
question = "What are you trained for?"
result = text_generation(question)
result

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "What are you trained for?\n\nI am a big fan of being a professional wrestler, though, and it's also fun having my own video game company where I don't need video game help. I have the whole world playing around with my"}]

# [Groq](https://wow.groq.com/)

## What is Groq ?  

   Groq is an open-source, distributed, and scalable graph database that allows users to store and query complex relationships between data entities. It's designed to handle large-scale graph data and provides a flexible and efficient way to store and query graph data.

## Is Groq free ?

   Groq is open-source, which means it is free to use, modify, and distribute. However, it's worth noting that Groq is still an actively developing project, and while it's free to use, it may not have the same level of support or resources as a commercial product. Additionally, while Groq is free, it may require additional infrastructure and resources to set up and maintain, depending on the scale and complexity of your use case.


try the Groq [playground](https://console.groq.com/playground)

In [ ]:
from langchain_groq.chat_models import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

def chat(question):
    llm = ChatGroq(temperature=0, groq_api_key=GROQ_API_KEY,
                   model_name="Llama3-8b-8192")
    system = "You are a helpful assistant."
    prompt = ChatPromptTemplate.from_messages(
        [("system", system), ("human", "{text}")])
    chain = prompt | llm
    response = chain.invoke({"text": question})
    return response


result = chat("1. What is Groq? 2.Is Groq Free ?")
print(result.content)

I'd be happy to help!

1. Groq is an open-source, distributed, and scalable graph database that allows users to store and query complex relationships between data entities. It's designed to handle large-scale graph data and provides a flexible and efficient way to store and query graph data. Groq is built on top of the Apache Arrow and Apache Parquet data formats, making it compatible with a wide range of data sources and tools.

2. Groq is open-source software, which means it is free to use, modify, and distribute. The project is maintained by a community of developers and contributors, and the source code is available on GitHub under the Apache 2.0 license. This means that users can use Groq without any licensing fees or restrictions, and they can also contribute to the project and help shape its development.

It's worth noting that while Groq is free and open-source, it may require some technical expertise to set up and use, especially for complex graph queries. However, the communi

In [ ]:
from langchain_groq.chat_models import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

async def async_chat(text):
    llm = ChatGroq(temperature=0, groq_api_key=GROQ_API_KEY,
                   model_name="Llama3-8b-8192")
    system = "You are a helpful assistant."
    human = "{text}"
    prompt = ChatPromptTemplate.from_messages(
        [("system", system), ("human", human)])
    chain = prompt | llm
    return chain.stream({"text": text})


for chunk in await async_chat({"text": """
                           What is Llama3 ? 
                           """}):
    print(chunk.content, end="", flush=True)

It seems like you're asking about Llama3! Llama3 is an open-source, cloud-based, and scalable machine learning platform developed by Google. It's designed to simplify the process of deploying and managing machine learning models in production environments.

Llama3 provides a range of features, including:

1. **Model serving**: Llama3 allows you to deploy and manage machine learning models in a scalable and efficient manner.
2. **Model management**: You can manage multiple models, track their performance, and update them as needed.
3. **Scalability**: Llama3 is designed to handle large volumes of data and scale to meet the needs of your application.
4. **Integration**: Llama3 integrates with various data sources, such as BigQuery, Cloud Storage, and more.

By using Llama3, you can focus on developing and improving your machine learning models, while the platform handles the underlying infrastructure and scalability.

Would you like to know more about Llama3 or is there something specifi

In [ ]:
for chunk in await async_chat({"text":"What are the latest date of your training data ?"}):
    print(chunk.content, end="", flush=True)

I was trained on a dataset that was current up to 2021. However, please note that my training data may not reflect any updates or changes that have occurred after that date. If you have any specific questions or topics you'd like to discuss, I'll do my best to provide you with accurate and helpful information.

# Open Web UI (100% local)

https://github.com/open-webui/open-webui

## Run with Docker

Ollama is running locally :  

`docker run -d --network host -v open-webui:/app/backend/data --name open-webui --restart always ghcr.io/open-webui/open-webui:main`

If Ollama is on a different Server:

`docker run -d --network=host -v open-webui:/app/backend/data -e OLLAMA_BASE_URL=http://127.0.0.1:11434 --name open-webui --restart always ghcr.io/open-webui/open-webui:main`


After installation, go to : http://localhost:8080

# The Fastest Solution to Install and Run (UI+LLM Speed)

## Msty.app + Groq API


Instructions: 

1. Go to website https://msty.app/ and download the msty UI
2. Go to https://wow.groq.com/, access the GroqCloud and create an API
3. Open Msty.app and configure with the Groq API key

## Anything LLM (Rag + Groq and other LLMs)


